In [6]:
import requests
from bs4 import BeautifulSoup
import csv

# List to store the game data
game_data = []

# Make request to page
for i in range(1663, 1715):  # replace range with your desired id range
    url = f'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023?phase={i}'
    response = requests.get(url)

    # Parse HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all game blocks
    game_blocks = soup.find_all('div', class_='block')

    # Extract game information from each block
    for block in game_blocks:
        date_time = block.find('div', class_='text-1').text.strip()
        team_1_img = block.find('div', class_='col-xs-5 p-t-10 nopadding').find('img')
        score_element = block.find('div', class_='text-3')
        team_2_img = block.find_all('div', class_='col-xs-5 p-t-10 nopadding')[1].find('img')

        # Check if elements exist before accessing their attributes
        if team_1_img and score_element and team_2_img:
            team_1 = team_1_img['title']
            score = score_element.text.strip()
            team_2 = team_2_img['title']

            # Append game data to the list
            game_data.append([date_time, team_1, score, team_2])

# Write game data to CSV file
with open('dataCBR2023raw.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Data', 'Casa', 'Resultado', 'Fora'])  # Write column headers
    writer.writerows(game_data)


In [2]:
import pandas as pd
df = pd.read_csv('dataCBR2023raw.csv')
df

,Data,Casa,Resultado,Fora
0,"Ter, 21/02/2023 19:00\r\n - GRUPO 1...",Marcílio Dias - SC,1 x 0,Chapecoense - SC
1,"Qua, 22/02/2023 16:00\r\n - GRUPO 2...",Falcon Futebol Clube - SE,1 x 3,Volta Redonda - RJ
2,"Qua, 22/02/2023 17:00\r\n - GRUPO 4...",Mutum Esporte Clube - MT,4 x 2,Londrina - PR
3,"Qua, 22/02/2023 19:00\r\n - GRUPO 3...",São Raimundo - RR,4 x 3,Cuiabá Saf - MT
4,"Qua, 22/02/2023 19:00\r\n - GRUPO 3...",Caucaia - CE,0 x 0,Tombense - MG
...,...,...,...,...
193,"Sáb, 15/07/2023 16:30\r\n -...",Corinthians - SP,(3)3 x 2(1),America Fc S.a.f. - MG
194,"Qua, 05/07/2023 21:30\r\n - GRU...",Flamengo - RJ,2 x 1,Athletico Paranaense - PR
195,"Qua, 12/07/2023 21:30\r\n -...",Athletico Paranaense - PR,0 x 2,Flamengo - RJ
196,Morumbi - Sao Paulo - SP\nDetalhes do jogo,São Paulo - SP,19:30,Corinthians - SP


In [9]:
#Cleans-up the 'Resultado' column
df['Resultado'] = df['Resultado'].str.replace(r'\([^)]*\)', '', regex=True)
# Drop rows where the result is only an "x"
df = df[df['Resultado'] != 'x']
df = df.reset_index(drop=True)

#finds and separates the Date in the 'Data' column
df['Data'] = df['Data'].str.extract(r'(\s\d+/\d+/\d+)')

# Split the "Resultado" column
new = df["Resultado"].str.split("x", n=1, expand=True)

# Checks if the split operation resulted in two columns
if new.shape[1] == 2:
    # Assign values to "Casa" and "Fora" columns
    df['GC'] = new[0]
    df['GF'] = new[1]
    
df[['GC', 'GF']] = df[['GC', 'GF']].apply(pd.to_numeric, errors='coerce')
# Drop the "Resultado" column
df = df.drop(['Resultado'], axis=1)


#changes dtype to datetime in first dataframe
df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
#creates a new column with the for the competition name

df = df.assign(Comp='CBR')


#creates new column with competition name

#comparing scores to display the results in the second dataframe
df.loc[df['GC'] == df['GF'], 'Res'] = 'E'
df.loc[df['GC'] > df['GF'], 'Res'] = 'VC' 
df.loc[df['GC'] < df['GF'], 'Res'] = 'VV'
# create a new column to store the season information in the second data frame
df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
df['Temp'] = df['Data'].dt.year
#organizes de columns of the sacond dataframe
new_order = ['Comp','Temp','Data','Casa','GC', 'GF','Fora','Res']
df= df.reindex(columns=new_order)


#df.replace({             }, inplace=True)

#drops unvalid data
df.dropna(axis=0, how='all', inplace=True)
#sort values by date
df = df.sort_values(by='Data')

#atributes a pontuation to each game
df['PC'] = df.apply(lambda x: 3 if x['Res'] == 'VC' else
                              1 if x['Res'] == 'E' else 0, axis=1)
df['PF'] = df.apply(lambda x: 3 if x['Res'] == 'VV' else
                              1 if x['Res'] == 'E' else 0, axis=1)

#atributes a number of games for each season
df['J'] = df.groupby('Temp').cumcount() + 1
#corrects any possible issue before saving as new file
df = df.dropna()



KeyError: 'Resultado'

In [4]:
import numpy as np


# Define the list of values to keep the '-'
condition = ['América - RN',
             'Atlético - GO', 
             'Atlético - BA', 
             'Botafogo - RJ', 
             'Botafogo - SP',
             'Botafogo - PB',
             'Fluminense - RJ',
             'Fluminense - PI',
             'Operário - MS',
             'Operário - PR',
             'União - MT']

def process_value(value):
    if value in condition:
        return value  # Keep the original value
    else:
        value = value.split('-', 1)[0].strip()  # Remove everything after '-' and strip whitespace
        value = value.replace('Saf', '').replace('S.a.f', '').replace('Futebol Clube', '').replace('Esporte Clube', '').strip()  # Remove 'Saf' and 'S.a.f' and strip whitespace
        return value

# Apply the function to the 'Casa' column
df['Casa'] = np.where(df['Casa'].isin(condition), df['Casa'], df['Casa'].apply(process_value))

# Apply the function to the 'Fora' column
df['Fora'] = np.where(df['Fora'].isin(condition), df['Fora'], df['Fora'].apply(process_value))


#Renames the teams 

df['Casa'] = df['Casa'].str.strip()
df['Fora'] = df['Fora'].str.strip()
df.replace({'Abc': 'ABC',
            'Asa': 'ASA',
            'Aguia de Maraba': 'Águia de Marabá',
            'América - RN': 'América-RN',
            'America Fc .': 'América-MG',
            'Atlético Mineiro': 'Atlético-MG',
            'Athletico Paranaense': 'Athletico-PR',
            'Atlhetic Ce': 'Atlhetic-MG',
            'Atlético - GO': 'Atlético-GO', 
            'Atlético - BA': 'Atlético-BA', 
            'Botafogo - RJ': 'Botafogo', 
            'Botafogo - SP': 'Botafogo-SP',
            'Botafogo - PB': 'Botafogo-PB',
            'Brasil': 'Brasil-RS',
            'Camboriu': 'Camboriú',
            'Crb': 'CRB',
            'Csa': 'CSA',
            'Ec Democrata': 'Democrata-MG',
            'Fluminense - RJ': 'Fluminense',
            'Fluminense - PI': 'Fluminense-PI',
            'Marilia': 'Marília',
            'Operário - MS': 'Operário-MS',
            'Operário - PR': 'Operário-PR',
            'União - MT': 'União-MT',
            'Real Noroeste Capixaba F. C.': 'Real Noroeste',
            'Sao Bernardo Fc': 'São Bernardo', 
            'Sao Francisco': 'São Francisco',
            'Vasco da Gama': 'Vasco',
            'Vitoria F. C.': 'Vitória-ES',
            'Ypiranga': 'Ypiranga-RS',
            'Vila Nova': 'Vila Nova-GO'
            
                        }, inplace=True)

In [11]:
df['Data'] = df['Data'].dt.strftime('%d-%m-%Y')
df.tail(20)

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
164,CBR,2023.0,17-05-2023,Grêmio,1.0,1.0,Cruzeiro,E,1,1,97.0
166,CBR,2023.0,17-05-2023,Sport,0.0,2.0,São Paulo,VV,0,3,98.0
172,CBR,2023.0,17-05-2023,Atlético-MG,2.0,0.0,Corinthians,VC,3,0,99.0
168,CBR,2023.0,17-05-2023,América-MG,2.0,0.0,Internacional,VC,3,0,100.0
173,CBR,2023.0,31-05-2023,Corinthians,2.0,0.0,Atlético-MG,VC,3,0,101.0
161,CBR,2023.0,31-05-2023,Bahia,1.0,1.0,Santos,E,1,1,102.0
163,CBR,2023.0,31-05-2023,Fortaleza,1.0,0.0,Palmeiras,VC,3,0,103.0
165,CBR,2023.0,31-05-2023,Cruzeiro,0.0,1.0,Grêmio,VV,0,3,104.0
169,CBR,2023.0,31-05-2023,Internacional,3.0,1.0,América-MG,VC,3,0,105.0
171,CBR,2023.0,31-05-2023,Botafogo,1.0,0.0,Athletico-PR,VC,3,0,106.0


In [52]:
unique_values = df['Casa'].unique()
unique_values_sorted = sorted(unique_values)
print(unique_values_sorted)

['ABC', 'ASA', 'América-MG', 'Athletico-PR', 'Atlhetic-MG', 'Atlético-BA', 'Atlético-GO', 'Atlético-MG', 'Bahia', 'Bahia de Feira', 'Botafogo', 'Botafogo-SP', 'Brasil-RS', 'CRB', 'CSA', 'Caldense', 'Camboriú', 'Campinense', 'Caucaia', 'Ceilândia', 'Cordino', 'Corinthians', 'Coritiba', 'Cruzeiro', 'Democrata-MG', 'Falcon', 'Flamengo', 'Fluminense', 'Fluminense-PI', 'Fortaleza', 'Grêmio', 'Humaitá', 'Iguatu', 'Internacional', 'Ituano', 'Jacuipense', 'Marcílio Dias', 'Maringá  .', 'Marília', 'Mutum', 'Nova Iguaçu', 'Náutico', 'Operário-MS', 'Palmeiras', 'Parnahyba', 'Paysandu', 'Princesa do Solimões', 'Real Ariquemes', 'Real Noroeste', 'Remo', 'Resende', 'Retrô', 'Santos', 'Sergipe', 'Sport', 'São Bernardo', 'São Francisco', 'São Luiz', 'São Paulo', 'São Raimundo', 'Tocantinópolis', 'Tombense', 'Trem', 'Tuna Luso', 'Tuntum', 'União-MT', 'Vasco da Gama .', 'Vitória-ES', 'Volta Redonda', 'Ypiranga-RS', 'Águia de Marabá']


In [12]:
df.to_csv('dataCBR2023.csv', index=False)


In [44]:
df

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
0,CBR,2023,2023-02-21,Marcílio Dias,1,0,Chapecoense,VC,3,0,1
1,CBR,2023,2023-02-22,Falcon,1,3,Volta Redonda,VV,0,3,2
2,CBR,2023,2023-02-22,Mutum,4,2,Londrina,VC,3,0,3
3,CBR,2023,2023-02-22,São Raimundo,4,3,Cuiabá,VC,3,0,4
4,CBR,2023,2023-02-22,Caucaia,0,0,Tombense,E,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...
110,CBR,2023,2023-07-05,São Paulo,1,0,Palmeiras,VC,3,0,112
109,CBR,2023,2023-07-12,Grêmio,1,1,Bahia,E,1,1,113
115,CBR,2023,2023-07-12,Athletico-PR,0,2,Flamengo,VV,0,3,114
111,CBR,2023,2023-07-13,Palmeiras,1,2,São Paulo,VV,0,3,115


In [54]:
unique_values = df['Fora'].unique()
unique_values_sorted = sorted(unique_values)
print(unique_values_sorted)

['ABC', 'América-MG', 'América-RN', 'Athletico-PR', 'Atlético-GO', 'Atlético-MG', 'Avaí', 'Bahia', 'Botafogo', 'Botafogo-PB', 'Botafogo-SP', 'Brasil-RS', 'Brasiliense', 'Brusque', 'CRB', 'CSA', 'Ceará', 'Chapecoense', 'Corinthians', 'Coritiba', 'Criciúma', 'Cruzeiro', 'Cuiabá', 'Ferroviário', 'Flamengo', 'Fluminense', 'Fortaleza', 'Goiás', 'Grêmio', 'Iguatu', 'Internacional', 'Ituano', 'Juventude', 'Londrina', 'Manaus', 'Marcílio Dias', 'Maringá  .', 'Mutum', 'Nova Iguaçu', 'Náutico', 'Operário-MS', 'Operário-PR', 'Palmeiras', 'Paysandu', 'Ponte Preta', 'Red Bull Bragantino', 'Remo', 'Retrô', 'Sampaio Corrêa', 'Santa Cruz', 'Santos', 'Sport', 'São Luiz', 'São Paulo', 'São Raimundo', 'Tombense', 'Vasco da Gama .', 'Vila Nova-GO', 'Vitória', 'Volta Redonda', 'Ypiranga-RS', 'Águia de Marabá']
